<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/M_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_G_inc.csv'  'My Drive'
'Colab Notebooks'	 G_inc_model.pt    P_inc_model.pt
'답안 작성용 파일.csv'	 I_ori_model.pt    S_ori_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [31]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='M')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000045,M,73,732,제품(자동차 가구 수공구등),시각전달매체,그래픽 디자인
1,id_0000062,M,71,711,사법기관에제출하는서류경매공매,의뢰인을 대리하여,등기 경매공매등신청대및대리상담
2,id_0000068,M,72,721,현장에서,토목,공사
3,id_0000075,M,72,729,사무실에서,관련업체 요구에 의해,"일반측량, 설계"
4,id_0000113,M,71,712,세무사업,의뢰를받아서,세무상담. 기장대리
...,...,...,...,...,...,...,...
28429,id_0999812,M,72,721,사무실에서,관련업체 요구에 의해,도시계획 및 조경설계
28430,id_0999818,M,73,739,사업장에서,고객의뢰를받아,무게계량
28431,id_0999820,M,71,712,세무사업,의뢰를받아서,기장대리. 세무상담
28432,id_0999876,M,71,711,사법기관에 제출하는 서류 경매 공매,의뢰인을 대리하여,등기 경매 공매등 선청대행및 대리 상담


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000045,M,73,732,제품(자동차 가구 수공구등) 시각전달매체 그래픽 디자인
1,id_0000062,M,71,711,사법기관에제출하는서류경매공매 의뢰인을 대리하여 등기 경매공매등신청대및대리상담
2,id_0000068,M,72,721,현장에서 토목 공사
3,id_0000075,M,72,729,"사무실에서 관련업체 요구에 의해 일반측량, 설계"
4,id_0000113,M,71,712,세무사업 의뢰를받아서 세무상담. 기장대리
...,...,...,...,...,...
28429,id_0999812,M,72,721,사무실에서 관련업체 요구에 의해 도시계획 및 조경설계
28430,id_0999818,M,73,739,사업장에서 고객의뢰를받아 무게계량
28431,id_0999820,M,71,712,세무사업 의뢰를받아서 기장대리. 세무상담
28432,id_0999876,M,71,711,사법기관에 제출하는 서류 경매 공매 의뢰인을 대리하여 등기 경매 공매등 선청대행및 ...


In [9]:
df['digit_3'].value_counts()

721    5513
711    3963
715    3318
712    3128
713    3025
733    2335
732    1743
729    1635
701    1538
731    1049
739     559
702     425
716     106
714      97
Name: digit_3, dtype: int64

## **# 데이터 증분 및 맞춤법 교정**

In [8]:
# 맞춤법 검사

In [10]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-61_tqi6g
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-61_tqi6g
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=74367588f27080502581473f85c0a31565ae080bf403e1e19084e1816b670de8
  Stored in directory: /tmp/pip-ephem-wheel-cache-n8ga9717/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [11]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [12]:
df.to_csv('df_M_spellcheck.csv', index=False, encoding='utf-8-sig')

In [17]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000045,M,73,732,제품(자동차 가구 수 공구 등) 시각전달 매체 그래픽 디자인
1,id_0000062,M,71,711,사법기관에 제출하는 서류 경매 공매 의뢰인을 대리하여 등기 경매 공매 등 신청대 및...
2,id_0000068,M,72,721,현장에서 토목 공사
3,id_0000075,M,72,729,"사무실에서 관련 업체 요구에 의해 일반측량, 설계"
4,id_0000113,M,71,712,세무사업 의뢰를 받아서 세무 상담. 기장대리
...,...,...,...,...,...
28429,id_0999812,M,72,721,사무실에서 관련 업체 요구에 의해 도시계획 및 조경설계
28430,id_0999818,M,73,739,사업장에서 고객 의뢰를 받아 무게 계량
28431,id_0999820,M,71,712,세무사업 의뢰를 받아서 기장대리. 세무 상담
28432,id_0999876,M,71,711,사법기관에 제출하는 서류 경매 공매 의뢰인을 대리하여 등기 경매 공매 등 신청 대행...


In [14]:
df['digit_3'].value_counts()

721    5513
711    3963
715    3318
712    3128
713    3025
733    2335
732    1743
729    1635
701    1538
731    1049
739     559
702     425
716     106
714      97
Name: digit_3, dtype: int64

In [18]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [20]:
df1=df[df['digit_3']==701] 
df2=df[df['digit_3']==702] 
df3=df[df['digit_3']==711] 
df4=df[df['digit_3']==712] 
df5=df[df['digit_3']==713] 
df6=df[df['digit_3']==714] 
df7=df[df['digit_3']==715] 
df8=df[df['digit_3']==716] 
df9=df[df['digit_3']==721] 
df10=df[df['digit_3']==729] 
df11=df[df['digit_3']==731] 
df12=df[df['digit_3']==732] 
df13=df[df['digit_3']==733] 
df14=df[df['digit_3']==739] 

df100=pd.concat([df9, df3, df4, df5, df7])
df200=pd.concat([df13, df12, df10, df1, df11])
df300=pd.concat([df14, df2])
df400=pd.concat([df8, df6])

df100=df100.reset_index(drop=True) #*0
df200=df200.reset_index(drop=True) #*3 
df300=df300.reset_index(drop=True) #*10
df400=df400.reset_index(drop=True) #*50

In [21]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df200['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df200['AI_id'][n]), 'p'+str(df200['AI_id'][n]), 'p'+str(df200['AI_id'][n])]
  digit_1_list=[df200['digit_1'][n], df200['digit_1'][n], df200['digit_1'][n]]
  digit_2_list=[df200['digit_2'][n], df200['digit_2'][n], df200['digit_2'][n]]
  digit_3_list=[df200['digit_3'][n], df200['digit_3'][n], df200['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df200_to_insert = pd.DataFrame(data_to_insert)
  df200= df200.append(df200_to_insert, ignore_index=True)
  n=n+1

In [22]:
df200

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000571,M,73,733,"광고 회사, 출판업자를 상대로 하는 사진관 운영"
1,id_0004950,M,73,733,사무실에서 고객 의뢰로 인물사진 작업
2,id_0005752,M,73,733,매장에서 사진 인화
3,id_0007303,M,73,733,관공서 의뢰를 받아서 사업장에서 항공촬영
4,id_0007527,M,73,733,다다 사진관에서 일반 고객을 대상으로 사진 촬영
...,...,...,...,...,...
33195,pid_0954332,M,73,731,사업장에서 가축을 대상으로 수의 및 축산동물 질병예방서비스
33196,pid_0954332,M,73,731,사업장에서 가축을 대상으로 수의 및 축산동물 질병예방서비스
33197,pid_0957082,M,73,731,고양이 치료 받아 동물병원에서 강아지. 일반인의 등 의뢰를
33198,pid_0957082,M,73,731,일반인의 의뢰를 받아 동물병원에서 강아지. 고양이 등 치료


In [23]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df300['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=10)
  id_list=['p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]),'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]),'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n])]
  digit_1_list=[df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n],df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n],df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n]]
  digit_2_list=[df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n],df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n],df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n]]
  digit_3_list=[df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n],df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n],df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df300_to_insert = pd.DataFrame(data_to_insert)
  df300= df300.append(df300_to_insert, ignore_index=True)
  n=n+1

In [24]:
df300

,AI_id,digit_1,digit_2,digit_3,text
0,id_0001327,M,73,739,"사무실에서 고객 의뢰로 연예인관리,매니저업"
1,id_0004794,M,73,739,다국어 번역으로 사무실 번역 서비스
2,id_0005425,M,73,739,정부의 지침에 의거 정부 에너지 절약 정책사업 열사용 기자재 검사 및 에너지 효율 ...
3,id_0008116,M,73,739,사업장에서 고객 의뢰를 받아 번역 서비스
4,id_0009740,M,73,739,사업장에서 공인 대리 관리 매너저업
...,...,...,...,...,...
11803,pid_0999504,M,70,702,사업장에서 예약에
11804,pid_0999504,M,70,702,사업장에서 예약에 의해 심리상담
11805,pid_0999504,M,70,702,사업장에서 예약에 의해 심리상담
11806,pid_0999504,M,70,702,사업장에서 심리상담


In [25]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df400['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=49)
  id_list=['p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]), 'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n]),'p'+str(df400['AI_id'][n])]
  digit_1_list=[df400['digit_1'][n], df400['digit_1'][n] , df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n], df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n],df400['digit_1'][n]]
  digit_2_list=[df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n], df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n],df400['digit_2'][n]]
  digit_3_list=[df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n] , df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n], df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n],df400['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df400_to_insert = pd.DataFrame(data_to_insert)
  df400= df400.append(df400_to_insert, ignore_index=True)
  n=n+1

In [26]:
df400

,AI_id,digit_1,digit_2,digit_3,text
0,id_0006008,M,71,716,정부 재정 지원 사업 종합 관리 산학협력단에서 계약 체결에 의해 입주업체 업무지원
1,id_0011367,M,71,716,"정부 프로젝트를 따서 시행 사무실에서, 국고지원 사업과 산학협력지원"
2,id_0016982,M,71,716,"해수부 관련 사무실에서 어촌개발 지원, 사업평가관리"
3,id_0045822,M,71,716,고객의 요청으로 연구과제를 사무지원 중재 및 공공관계 서비스
4,id_0069259,M,71,716,다른 사업체에게 경영 문제 자문 지원 학교 서비스 기업 지원 서비스
...,...,...,...,...,...
10348,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
10349,pid_0998381,M,71,714,가지고 영업하는 대상 여론조사 마케팅 영역
10350,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
10351,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"


기존 데이터 합치기

In [27]:
df=pd.concat([df100, df200, df300, df400])
df.reset_index(drop=True)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000068,M,72,721,현장에서 토목 공사
1,id_0000440,M,72,721,"사무실에서 계약의뢰에 의해 건축설계, 감리"
2,id_0000562,M,72,721,건축사 사무소에서 고객의 의뢰에 따라 건축설계
3,id_0000910,M,72,721,사무실에서 관련 업체 요구에 의해 토지개발 예산 산출 컨설팅
4,id_0000934,M,72,721,사무실에서 관련 업체 요구에 의해 환경영향평가 보고서 제공
...,...,...,...,...,...
74303,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
74304,pid_0998381,M,71,714,가지고 영업하는 대상 여론조사 마케팅 영역
74305,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
74306,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"


In [28]:
df['digit_3'].value_counts()

733    9340
732    6972
739    6708
729    6540
701    6152
721    5513
716    5406
702    5100
714    4947
731    4196
711    3963
715    3318
712    3128
713    3025
Name: digit_3, dtype: int64

In [29]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000068,M,72,721,현장에서 토목 공사
1,id_0000440,M,72,721,"사무실에서 계약의뢰에 의해 건축설계, 감리"
2,id_0000562,M,72,721,건축사 사무소에서 고객의 의뢰에 따라 건축설계
3,id_0000910,M,72,721,사무실에서 관련 업체 요구에 의해 토지개발 예산 산출 컨설팅
4,id_0000934,M,72,721,사무실에서 관련 업체 요구에 의해 환경영향평가 보고서 제공
...,...,...,...,...,...
10348,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
10349,pid_0998381,M,71,714,가지고 영업하는 대상 여론조사 마케팅 영역
10350,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
10351,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"


In [30]:
df.to_csv('df_M_inc.csv', index=False, encoding='utf-8-sig')